In [1]:
import torch, torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import StepLR

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#device = 'cpu'
print(device)

cpu


In [3]:
# define input size, hidden layer size, output size
D_i, D_k, D_o = 10, 40, 5
# create model with two hidden layers
model = nn.Sequential(
       nn.Linear(D_i, D_k),
       nn.ReLU(),
       nn.Linear(D_k, D_k),
       nn.ReLU(),
       nn.Linear(D_k, D_o)).to(device)

In [4]:
# He initialization of weights
def weights_init(layer_in):
    if isinstance(layer_in, nn.Linear):
        nn.init.kaiming_uniform(layer_in.weight)
        #layer_in.weight.data.fill_(-10.0)
        #layer_in.bias.data.fill_(-100.0)
model.apply(weights_init)

/tmp/ipykernel_2574/3350313036.py:4: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  nn.init.kaiming_uniform(layer_in.weight)


Sequential(
  (0): Linear(in_features=10, out_features=40, bias=True)
  (1): ReLU()
  (2): Linear(in_features=40, out_features=40, bias=True)
  (3): ReLU()
  (4): Linear(in_features=40, out_features=5, bias=True)
)

In [5]:
# choose least squares loss function
criterion = nn.MSELoss()
# construct SGD optimizer and initialize learning rate and momentum
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1, momentum=0.9)
# object that decreases learning rate by half every 10 epochs
scheduler = StepLR(optimizer, step_size=10, gamma=0.5)

In [6]:
# create 100 random data points and store in data loader class
x = torch.randn(100, D_i).to(device)
y = torch.randn(100, D_o).to(device)
data_loader = DataLoader(TensorDataset(x,y), batch_size=10, shuffle=True)

In [7]:
# loop over the dataset 100 times
def train():
    for epoch in range(10):
        epoch_loss = 0.0
        # loop over batches
        for i, data in enumerate(data_loader):
            # retrieve inputs and labels for this batch
            x_batch, y_batch = data
            # zero the parameter gradients
            optimizer.zero_grad()
            # forward pass
            pred = model(x_batch)
            loss = criterion(pred, y_batch)
            # backward pass
            loss.backward()
            # SGD update
            optimizer.step()
            # update statistics
            epoch_loss += loss.item()
        # print error
        print(f'Epoch {epoch:5d}, loss {epoch_loss:.3f}')
        # tell scheduler to consider updating learning rate
        scheduler.step()

In [8]:
%timeit train()

Epoch     0, loss 19.730
Epoch     1, loss 10.741
Epoch     2, loss 10.073
Epoch     3, loss 9.179
Epoch     4, loss 8.214
Epoch     5, loss 8.473
Epoch     6, loss 8.160
Epoch     7, loss 7.749
Epoch     8, loss 7.785
Epoch     9, loss 7.400
Epoch     0, loss 6.498
Epoch     1, loss 5.971
Epoch     2, loss 5.607
Epoch     3, loss 5.130
Epoch     4, loss 4.853
Epoch     5, loss 4.614
Epoch     6, loss 4.438
Epoch     7, loss 4.295
Epoch     8, loss 4.276
Epoch     9, loss 4.130
Epoch     0, loss 3.899
Epoch     1, loss 3.808
Epoch     2, loss 3.738
Epoch     3, loss 3.683
Epoch     4, loss 3.659
Epoch     5, loss 3.641
Epoch     6, loss 3.604
Epoch     7, loss 3.581
Epoch     8, loss 3.551
Epoch     9, loss 3.547
Epoch     0, loss 3.489
Epoch     1, loss 3.478
Epoch     2, loss 3.480
Epoch     3, loss 3.464
Epoch     4, loss 3.462
Epoch     5, loss 3.450
Epoch     6, loss 3.442
Epoch     7, loss 3.446
Epoch     8, loss 3.426
Epoch     9, loss 3.413
Epoch     0, loss 3.408
Epoch     1, 